<a href="https://colab.research.google.com/github/soniajoseph/ExcaliburML/blob/master/Gaussian_Mixture_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np  
from scipy.stats import multivariate_normal

In [0]:
class GMM():
  def __init__(self, k, iterations=1000):
    self.k = k
    self.iterations = iterations
    self.phi_history = []
  
  def initialize(self):
    self.m, self.n = self.X.shape
    # Initialize means as random X values
    random_num = np.random.randint(0, high=self.m, size=self.k)
    self.means = np.asarray([self.X[i,:] for i in random_num])
    # Initialize covariances 
    self.sigma = [np.cov(self.X.T) for _ in range(self.k)]
    self.weights = np.full((self.m, k), fill_value=1/k)
    self.phi = np.full((self.m, k), fill_value=1/k)

  def expectation(self):
    # Hold mean and variance fixed. Calculate weights of each data point.
    probs = np.zeros((self.m, k))
    for i in range(self.k):
      mv = multivariate_normal(mean=self.means[i], cov=self.sigma[i])
      probs[:,i] = mv.pdf(self.X)
    n = probs * self.phi
    self.weights = n / np.sum(n, axis=1).reshape(-1,1)
    # Calculate weight of each Gaussian.
    self.phi = np.mean(self.weights, axis=0)
    self.phi_history.append(self.phi)

  def maximization(self):
    # Hold weights and phi fixed. Calculate mean and variance.
    for i in range(self.k):
      # Calculate mean
      self.means[i] = (self.weights[:,i].reshape(-1,1).T @ self.X) / self.m 
      aweights = (self.weights[:,i]/np.sum(self.weights[:,i], axis=0)).flatten() # get probabilities of weights
      
      # Calculate covariance
      x = self.X - self.means[i, :] # (N x d)
      x_mu = np.matrix(x)
      gamma_diag = np.matrix(np.diag(self.weights[:,i]))
      sigma_i = x.T * gamma_diag * x
      self.sigma[i]=(sigma_i) / np.sum(self.weights, axis = 0)[:,np.newaxis][i]
  
  def fit(self, X):
    self.X = X
    self.initialize()
    for i in range(self.iterations):
      self.expectation()
      self.maximization()

  def predict(self, Xtest):
    p = np.ones((Xtest.shape[0],k))
    yhat = []
    for i in range(k):
      p[:,i] = multivariate_normal(self.means[i], self.sigma[i]).pdf(Xtest) * self.phi[i]
    yhat.append(np.argmax(p,axis=1))
    return yhat[0]

  def accuracy(self, yhat, Ytest):
    return (yhat == Ytest).sum() / len(yhat)


In [0]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [0]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target 
X = preprocessing.scale(X)